In [2]:
#!/usr/bin/env python3
import sevenbridges as sbg
import sys
import re
import os
import concurrent.futures
from requests import request
from datetime import datetime
config = sbg.Config(profile='cavatica')
api = sbg.Api(config=config)
pnoc_wes = 'kfdrc-harmonization/sd-m3dbxd12'

### Get files for fq2cram2gvcf (germline) run

In [2]:
def get_relevant_file_objs(api, project):
    # will hold all fastq objects for inputs
    fq_dict = {}
    # will hold all keys for reference inputs
    ref_fn_dict = {}
    ref_fn_dict['Homo_sapiens_assembly38.contam.bed'] = 'contamination_sites_bed'
    ref_fn_dict['Homo_sapiens_assembly38.contam.mu'] = 'contamination_sites_mu'
    ref_fn_dict['Homo_sapiens_assembly38.contam.UD'] = 'contamination_sites_ud'
    ref_fn_dict['Homo_sapiens_assembly38.dbsnp138.vcf'] = 'dbsnp_vcf'
    ref_fn_dict['Homo_sapiens_assembly38.fasta'] = 'indexed_reference_fasta'
    ref_fn_dict['1000G_omni2.5.hg38.vcf.gz'] = 'knownsites'
    ref_fn_dict['1000G_phase1.snps.high_confidence.hg38.vcf.gz'] = 'knownsites'
    ref_fn_dict['Homo_sapiens_assembly38.known_indels.vcf.gz'] = 'knownsites'
    ref_fn_dict['Mills_and_1000G_gold_standard.indels.hg38.vcf.gz'] = 'knownsites'
    ref_fn_dict['Homo_sapiens_assembly38.dict'] = 'reference_dict'
    ref_fn_dict['Strexome_targets_sortedTabs2_padded150_38.bed.list.interval'] = 'intervals'
    ref_fn_dict['wgs_calling_regions.hg38.interval_list'] = 'wgs_calling_interval_list'
    ref_fn_dict['wgs_evaluation_regions.hg38.interval_list'] = 'wgs_evaluation_interval_list'
    # will hold all input reference objects
    ref_obj_dict = {}
    ref_obj_dict['knownsites'] = []
    all_files = api.files.query(project=project).all()
    for file_obj in all_files:
        fname = file_obj.name
        if 'fastq.gz' in fname:
            fq_dict[fname] = file_obj
            sys.stderr.write('Fastq candidate found:' + fname + '\n')
        elif fname in ref_fn_dict:
            if ref_fn_dict[fname] == 'knownsites':
                ref_obj_dict['knownsites'].append(file_obj)
            else:
                ref_obj_dict[ref_fn_dict[fname]] = file_obj
            sys.stderr.write('Reference found:' + fname + '\n')
    return fq_dict, ref_obj_dict
    

### Get fq2cram only refs

In [3]:
def get_fq2cram_ref_objs(api, project):
    # will hold all fastq objects for inputs
    fq_dict = {}
    # will hold all keys for reference inputs
    ref_fn_dict = {}
    ref_fn_dict['Homo_sapiens_assembly38.fasta'] = 'indexed_reference_fasta'
    ref_fn_dict['1000G_omni2.5.hg38.vcf.gz'] = 'knownsites'
    ref_fn_dict['1000G_phase1.snps.high_confidence.hg38.vcf.gz'] = 'knownsites'
    ref_fn_dict['Homo_sapiens_assembly38.known_indels.vcf.gz'] = 'knownsites'
    ref_fn_dict['Mills_and_1000G_gold_standard.indels.hg38.vcf.gz'] = 'knownsites'
    ref_fn_dict['Homo_sapiens_assembly38.dict'] = 'reference_dict'
    ref_fn_dict['Strexome_targets_sortedTabs2_padded150_38.bed.list.interval'] = 'targets_intervals'
    # will hold all input reference objects
    ref_obj_dict = {}
    ref_obj_dict['knownsites'] = []
    all_files = api.files.query(project=project).all()
    for file_obj in all_files:
        fname = file_obj.name
        if 'fastq.gz' in fname:
            fq_dict[fname] = file_obj
            sys.stderr.write('Fastq candidate found:' + fname + '\n')
        elif fname in ref_fn_dict:
            if ref_fn_dict[fname] == 'knownsites':
                ref_obj_dict['knownsites'].append(file_obj)
            else:
                ref_obj_dict[ref_fn_dict[fname]] = file_obj
            sys.stderr.write('Reference found:' + fname + '\n')
    return fq_dict, ref_obj_dict

### Create tasks for fq2cram only

In [4]:
def create_fq2cram_task(bs_id, cur_inputs, fq_dict, ref_dict, api, project):
    task_name = 'WESfq2cram2-' + bs_id
    app_name = project + '/kfdrc-alignment-fqinput-cram-only-wf'
    inputs = ref_dict
    inputs['rgs'] = []
    inputs['files_R1'] = []
    inputs['files_R2'] = []
    for rg in cur_inputs:
        inputs['rgs'].append(rg)
        inputs['files_R1'].append(fq_dict[cur_inputs[rg][0]])
        inputs['files_R2'].append(fq_dict[cur_inputs[rg][1]])
    task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=inputs, run=False)
    task.inputs['output_basename'] = task.id
    task.save()
    out_task.write('\t'.join((task_name, task.id)) + '\n')

### Create tasks for fq2cram2gvcf (germline) run

In [3]:
def create_task(bs_id, cur_inputs, fq_dict, ref_dict, api, project):
    task_name = 'WESfq2cram2gvcf-' + bs_id
    app_name = project + '/kf-alignment-fq-input-wf-wes'
    inputs = ref_dict
    inputs['rgs'] = []
    inputs['files_R1'] = []
    inputs['files_R2'] = []
    for rg in cur_inputs:
        inputs['rgs'].append(rg)
        inputs['files_R1'].append(fq_dict[cur_inputs[rg][0]])
        inputs['files_R2'].append(fq_dict[cur_inputs[rg][1]])
    task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=inputs, run=False)
    task.inputs['output_basename'] = task.id
    task.save()
    out_task.write('\t'.join((task_name, task.id)) + '\n')

In [5]:
def process_bs(bs_dict, bs_id, rg_csv, fq_csv):
    if bs_id != 'NA':
        if bs_id not in bs_dict:
            bs_dict[bs_id] = {}
        rgs = rg_csv.split(',')
        fqs = fq_csv.split(',')
        # sys.stderr.write(str(len(rgs)) + 'read groups and ' + str(len))
        for i in range(0, len(rgs), 1):
            j = (2 * i)         
            bs_dict[bs_id][rgs[i]] = []
            bs_dict[bs_id][rgs[i]].append(fqs[j])
            bs_dict[bs_id][rgs[i]].append(fqs[(j + 1)])
    return bs_dict

In [6]:
def process_input_info(info_fn):
    bs_dict = {}
    info_fh = open(info_fn)
    next(info_fh)
    for line in info_fh:
        info = line.rstrip('\n').split('\t')
        bs_dict = process_bs(bs_dict, info[1], info[2], info[3])
        bs_dict = process_bs(bs_dict, info[4], info[5], info[6])
    return bs_dict

## Set up jobs to all run fq2cram2gvcf

In [7]:
input_info = '/Users/brownm28/Documents/2018-Nov-6_open_dipg/pnoc_inputs.txt'
(fastqs, references) = get_relevant_file_objs(api, pnoc_wes)
bs_inputs = process_input_info(input_info)
x = 1
n = 5
out_task = open('/Users/brownm28/Documents/2018-Nov-6_open_dipg/pnoc_wes_align_tasks.txt', 'w')
for bs_id in bs_inputs:
    #if x % n == 0:
    #    sys.stderr.write('TEST MODE\n')
    #    break
    #x+=1
    create_task(bs_id, bs_inputs[bs_id], fastqs, references, api, pnoc_wes)
out_task.close()

Reference found:1000G_omni2.5.hg38.vcf.gz
Reference found:1000G_phase1.snps.high_confidence.hg38.vcf.gz
Fastq candidate found:C021_0001_20140916_BR_Whole_T3_A1STX_A04116_HAB2CADXX_AAACAT_L001_R1_001.fastq.gz
Fastq candidate found:C021_0001_20140916_BR_Whole_T3_A1STX_A04116_HAB2CADXX_AAACAT_L001_R2_001.fastq.gz
Fastq candidate found:C021_0001_20140916_BR_Whole_T3_A1STX_A04116_HAB2CADXX_AAACAT_L002_R1_001.fastq.gz
Fastq candidate found:C021_0001_20140916_BR_Whole_T3_A1STX_A04116_HAB2CADXX_AAACAT_L002_R2_001.fastq.gz
Fastq candidate found:C021_0001_20140916_BR_Whole_T3_TSMRU_A04100.proj.R1.fastq.gz
Fastq candidate found:C021_0001_20140916_BR_Whole_T3_TSMRU_A04100.proj.R2.fastq.gz
Fastq candidate found:C021_0001_20140916_PB_Whole_C1_A1STX_A04117_HAB2CADXX_CAAAAG_L001_R1_001.fastq.gz
Fastq candidate found:C021_0001_20140916_PB_Whole_C1_A1STX_A04117_HAB2CADXX_CAAAAG_L001_R2_001.fastq.gz
Fastq candidate found:C021_0001_20140916_PB_Whole_C1_A1STX_A04117_HAB2CADXX_CAAAAG_L002_R1_001.fastq.gz
Fa

Fastq candidate found:C021_0025_002724_BR_Whole_T2_K1STX_A15764_001_R1.fastq.gz
Fastq candidate found:C021_0025_002724_BR_Whole_T2_K1STX_A15764_001_R2.fastq.gz
Fastq candidate found:C021_0025_002724_BR_Whole_T2_TSMRU_A15745_000_R1.fastq.gz
Fastq candidate found:C021_0025_002724_BR_Whole_T2_TSMRU_A15745_000_R2.fastq.gz
Fastq candidate found:C021_0025_002724_BR_Whole_T2_TSMRU_A15745_001_R1.fastq.gz
Fastq candidate found:C021_0025_002724_BR_Whole_T2_TSMRU_A15745_001_R2.fastq.gz
Fastq candidate found:C021_0025_002724_PB_Whole_C1_K1STX_A15765_000_R1.fastq.gz
Fastq candidate found:C021_0025_002724_PB_Whole_C1_K1STX_A15765_000_R2.fastq.gz
Fastq candidate found:C021_0025_002724_PB_Whole_C1_K1STX_A15765_001_R1.fastq.gz
Fastq candidate found:C021_0025_002724_PB_Whole_C1_K1STX_A15765_001_R2.fastq.gz
Fastq candidate found:C021_0026_002831_BR_Whole_T2_K1STX_A16464_000_R1.fastq.gz
Fastq candidate found:C021_0026_002831_BR_Whole_T2_K1STX_A16464_000_R2.fastq.gz
Fastq candidate found:C021_0026_002831_B

In [9]:
def run_task(api, entry):
    info = entry.rstrip('\n').split()
    # (tag, tid) = info[-1].split(':')
    task = api.tasks.get(id=info[1])
    task.run()

In [10]:
task_fn = '/Users/brownm28/Documents/2018-Nov-6_open_dipg/pnoc_wes_align_tasks.txt'
with concurrent.futures.ThreadPoolExecutor(8) as executor:
    results = {executor.submit(run_task, api, entry): entry for entry in open(task_fn)}

## Run subset of BS IDs with fq2cram only pipe

In [9]:
input_info = '/Users/brownm28/Documents/2018-Nov-6_open_dipg/pnoc_inputs.txt'
# bs_to_run = '/Users/brownm28/Documents/2018-Nov-6_open_dipg/bs_id_cram-only_re-run.txt'
bs_to_run = '/Users/brownm28/Documents/2018-Nov-6_open_dipg/one_more_rpt.txt'
bs_run = {}
for line in open(bs_to_run):
    bs_run[line.rstrip('\n')] = 1
(fastqs, references) = get_fq2cram_ref_objs(api, pnoc_wes)
bs_inputs = process_input_info(input_info)
x = 1
n = 5
out_task = open('/Users/brownm28/Documents/2018-Nov-6_open_dipg/pnoc_wes_align_tasks.txt', 'a')
for bs_id in bs_inputs:
    #if x % n == 0:
    #    sys.stderr.write('TEST MODE\n')
    #    break
    #x+=1
    if bs_id in bs_run:
        create_fq2cram_task(bs_id, bs_inputs[bs_id], fastqs, references, api, pnoc_wes)
out_task.close()

Reference found:1000G_omni2.5.hg38.vcf.gz
Reference found:1000G_phase1.snps.high_confidence.hg38.vcf.gz
Fastq candidate found:C021_0001_20140916_BR_Whole_T3_A1STX_A04116_HAB2CADXX_AAACAT_L001_R1_001.fastq.gz
Fastq candidate found:C021_0001_20140916_BR_Whole_T3_A1STX_A04116_HAB2CADXX_AAACAT_L001_R2_001.fastq.gz
Fastq candidate found:C021_0001_20140916_BR_Whole_T3_A1STX_A04116_HAB2CADXX_AAACAT_L002_R1_001.fastq.gz
Fastq candidate found:C021_0001_20140916_BR_Whole_T3_A1STX_A04116_HAB2CADXX_AAACAT_L002_R2_001.fastq.gz
Fastq candidate found:C021_0001_20140916_BR_Whole_T3_TSMRU_A04100.proj.R1.fastq.gz
Fastq candidate found:C021_0001_20140916_BR_Whole_T3_TSMRU_A04100.proj.R2.fastq.gz
Fastq candidate found:C021_0001_20140916_PB_Whole_C1_A1STX_A04117_HAB2CADXX_CAAAAG_L001_R1_001.fastq.gz
Fastq candidate found:C021_0001_20140916_PB_Whole_C1_A1STX_A04117_HAB2CADXX_CAAAAG_L001_R2_001.fastq.gz
Fastq candidate found:C021_0001_20140916_PB_Whole_C1_A1STX_A04117_HAB2CADXX_CAAAAG_L002_R1_001.fastq.gz
Fa

Fastq candidate found:C021_0019_002532_XX_Whole_T3_TSMRU_A14592_000_R1.fastq.gz
Fastq candidate found:C021_0019_002532_XX_Whole_T3_TSMRU_A14592_000_R2.fastq.gz
Fastq candidate found:C021_0019_002532_XX_Whole_T3_TSMRU_A14592_001_R1.fastq.gz
Fastq candidate found:C021_0019_002532_XX_Whole_T3_TSMRU_A14592_001_R2.fastq.gz
Fastq candidate found:C021_0020_002534_PB_Whole_C1_K1STX_A14605_000_R1.fastq.gz
Fastq candidate found:C021_0020_002534_PB_Whole_C1_K1STX_A14605_000_R2.fastq.gz
Fastq candidate found:C021_0020_002534_PB_Whole_C1_K1STX_A14605_001_R1.fastq.gz
Fastq candidate found:C021_0020_002534_PB_Whole_C1_K1STX_A14605_001_R2.fastq.gz
Fastq candidate found:C021_0020_002534_XX_Whole_T2_K1STX_A14604_000_R1.fastq.gz
Fastq candidate found:C021_0020_002534_XX_Whole_T2_K1STX_A14604_000_R2.fastq.gz
Fastq candidate found:C021_0020_002534_XX_Whole_T2_K1STX_A14604_001_R1.fastq.gz
Fastq candidate found:C021_0020_002534_XX_Whole_T2_K1STX_A14604_001_R2.fastq.gz
Fastq candidate found:C021_0020_002534_X

## Delete extraneous files created from incorrect pipe run on tumor samples

In [14]:
tasks = api.tasks.query(project = pnoc_wes, status = 'COMPLETED').all()
input_info = open('/Users/brownm28/Documents/2018-Nov-6_open_dipg/pnoc_inputs.txt')
bs_dict = {}
next(input_info)
for line in input_info:
    info = line.rstrip('\n').split('\t')
    bs_dict[info[1]] = 0
out_rm = ('gvcf', 'gvcf_calling_metrics')
for task in tasks:
    check = task.name.split('-')
    if check[1] in bs_dict and check[0] == 'WESfq2cram2gvcf':
        sys.stderr.write('Found task outputs to cleanup for ' + task.name + '\n')
        print('Deleting ' + task.outputs[out_rm[0]].name)
        task.outputs[out_rm[0]].delete()
        for obj in task.outputs[out_rm[1]]:
            print('Deleting ' + obj.name)
            obj.delete()

Found task outputs to cleanup for WESfq2cram2gvcf-BS_GBT44HST


Deleting 1736d267-2b7e-4228-952f-907c845f54fd.g.vcf.gz
Deleting 1736d267-2b7e-4228-952f-907c845f54fd.variant_calling_detail_metrics
Deleting 1736d267-2b7e-4228-952f-907c845f54fd.variant_calling_summary_metrics
Deleting c3868405-b2aa-4140-a749-e1573412cc6a.g.vcf.gz


Found task outputs to cleanup for WESfq2cram2gvcf-BS_WYTDVC0Y
Found task outputs to cleanup for WESfq2cram2gvcf-BS_7KR13R3P


Deleting c3868405-b2aa-4140-a749-e1573412cc6a.variant_calling_detail_metrics
Deleting c3868405-b2aa-4140-a749-e1573412cc6a.variant_calling_summary_metrics
Deleting 98661fda-6eec-44ee-bb5a-d759aad6d705.g.vcf.gz
Deleting 98661fda-6eec-44ee-bb5a-d759aad6d705.variant_calling_detail_metrics
Deleting 98661fda-6eec-44ee-bb5a-d759aad6d705.variant_calling_summary_metrics
Deleting 0892df4b-b119-4264-99ac-e80df7ede0ec.g.vcf.gz
Deleting 0892df4b-b119-4264-99ac-e80df7ede0ec.variant_calling_detail_metrics
Deleting 0892df4b-b119-4264-99ac-e80df7ede0ec.variant_calling_summary_metrics


Found task outputs to cleanup for WESfq2cram2gvcf-BS_6DT506HY
Found task outputs to cleanup for WESfq2cram2gvcf-BS_GKXD33YH


...

Deleting 8ae8d807-e24a-4c3b-8225-a13cbb335a0e.g.vcf.gz
Deleting 8ae8d807-e24a-4c3b-8225-a13cbb335a0e.variant_calling_detail_metrics
Deleting 8ae8d807-e24a-4c3b-8225-a13cbb335a0e.variant_calling_summary_metrics
Deleting c85b11f5-98ff-4208-9340-ef1ac325f0cb.g.vcf.gz


Found task outputs to cleanup for WESfq2cram2gvcf-BS_QZPGVNBB
Found task outputs to cleanup for WESfq2cram2gvcf-BS_6687ZNGS


Deleting c85b11f5-98ff-4208-9340-ef1ac325f0cb.variant_calling_detail_metrics
Deleting c85b11f5-98ff-4208-9340-ef1ac325f0cb.variant_calling_summary_metrics
Deleting a244678e-f02f-4341-b1aa-bb7f42ec3063.g.vcf.gz
Deleting a244678e-f02f-4341-b1aa-bb7f42ec3063.variant_calling_detail_metrics
Deleting a244678e-f02f-4341-b1aa-bb7f42ec3063.variant_calling_summary_metrics
Deleting 96d4ad57-2a00-4dd1-bf8e-d69ecb1ba8e5.g.vcf.gz
Deleting 96d4ad57-2a00-4dd1-bf8e-d69ecb1ba8e5.variant_calling_detail_metrics
Deleting 96d4ad57-2a00-4dd1-bf8e-d69ecb1ba8e5.variant_calling_summary_metrics


Found task outputs to cleanup for WESfq2cram2gvcf-BS_2HDEZFTW
Found task outputs to cleanup for WESfq2cram2gvcf-BS_74TZJTW0


Deleting 81f1b5f3-9c8f-4046-b7fb-b19291c472f1.g.vcf.gz
Deleting 81f1b5f3-9c8f-4046-b7fb-b19291c472f1.variant_calling_detail_metrics
Deleting 81f1b5f3-9c8f-4046-b7fb-b19291c472f1.variant_calling_summary_metrics
